# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Satellite Imagery

BrainStation Winter 2023: Data Science

In [ ]:
# imports

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

In [ ]:
from usgs import api

# Set the EarthExplorer catalog
node = 'EE'# this indicates earth explorer website

# Set the Hyperion and Landsat 8 dataset
hyperion_dataset = 'EO1_HYP_PUB'
landsat8_dataset = 'LANDSAT_8'

# Set the scene ids
hyperion_scene_id = 'EO1H1820422014302110K2_SG1_01'
landsat8_scene_id = 'LC80290462015135LGN00'

# Submit requests to USGS servers
api.metadata(hyperion_dataset, node, [hyperion_scene_id])
api.metadata(landsat8_dataset, node, [landsat8_scene_id])

usgs.api.metadata(dataset, node, entityids, extended=False, api_key=None)

In [ ]:
# https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b

In [ ]:
from sentinelsat import SentinelAPI

user = 'sralter' 
password = 'rcy1qmr-btk1DFT1eba' 
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
import geopandas as gpd
import folium 

# santa monica bounds file named nReserve
nReserve = gpd.read_file('/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/shapefiles/santa_monica_bounds/sm_bounds.geojson')

# empty base map in Folium centered around the bounding box
m = folium.Map([34.08483,-118.70617], zoom_start=12)
folium.GeoJson(nReserve).add_to(m)
m

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in nReserve['geometry']:
    footprint = i

In [ ]:
products = api.query(footprint,
                     #date = ('20190601', '20190626'),
                     date = ('20220101', '20221231'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,15)
                    )

In [ ]:
products

In [ ]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted
products_gdf

In [ ]:
# api.download('becdf74b-fb47-4010-84f1-2c271a501266')

In [ ]:
import rasterio

R10 = '/Users/sra/Files/brainstation_2023_ds_capstone/brainstation_2023_ds_capstone/01_capstone_data/notebooks/S2A_MSIL2A_20221224T183801_N0509_R027_T11SLT_20221224T213852.SAFE/GRANULE/L2A_T11SLT_A039206_20221224T184116/IMG_DATA/R10m'
b4 = rio.open(R10+'/T11SLT_20221224T183801_B04_10m.jp2')
b3 = rio.open(R10+'/T11SLT_20221224T183801_B03_10m.jp2')
b2 = rio.open(R10+'/T11SLT_20221224T183801_B02_10m.jp2')


# Create an RGB image 
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()

# Clip raster to polygon

In [ ]:
import shapely

In [ ]:
shapely.geos.geos_version

In [ ]:
# first, merge perimeters
# https://stackoverflow.com/questions/61035170/merging-polygon-shapefiles-in-python

# this was done in QGIS for now. View merged perimeters, 
# to confirm only one polygon in the dataset
sm_perim_merge=gpd.read_file('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson')
sm_perim_merge

In [ ]:
# get coordinates from geometry of sm_perim_merge

sm_perim_merge['geometry']

In [ ]:
import gdal

gdal.UseExceptions()

In [ ]:
# define filenames

# raster in
rasin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'
# CRS:
# Coordinate Reference System (CRS)
# Name				EPSG:26911 - NAD83 / UTM zone 11N
# Units				meters
# Method			Universal Transverse Mercator (UTM)
# Celestial body	Earth
# Reference			Static (relies on a datum which is plate-fixed)


# shapefile in
shpin='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge_shp.shp'
# I reprojected the shapefile to the same crs as the raster above in QGIS
# Coordinate Reference System (CRS)

# Name
# EPSG:26911 - NAD83 / UTM zone 11N
# Units
# meters
# Method
# Universal Transverse Mercator (UTM)
# Celestial body
# Earth
# Reference
# Static (relies on a datum which is plate-fixed)


# raster out
rasout='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_sm_fire.tif'

#                 out     in
result=gdal.Warp(rasout,rasin,cutlineDSName=shpin)

iface.addRasterLayer(rasout)

In [ ]:
# https://opensourceoptions.com/blog/how-to-clip-a-raster-to-a-polygon-or-an-extent-with-python-extract-by-mask/

import gdal

fn_in = r"/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif"
fn_clip = '/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_sm_fire.tif'
fn_poly = r"path/to/polygon.gpkg"

# gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)

# from osgeo import gdal
# fn_in = r"..\..\course-gdal-python\data\input\USGS_one_meter_x64y486_ID_FEMAHQ_2018.tif"
fn_in=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'

# fn_poly = r"..\..\course-gdal-python\data\input\polygon.gpkg"
fn_poly=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson'

# fn_clip = '../data/output/clip_polygon.tif'
fn_clip='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip_fire.tif'


ortho_fire=gdal.Warp(fn_clip, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True)
ortho_fire
# gdal.Warp(fn_clip2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=False)
# where = "ID = '1'"
# gdal.Warp(fn_clip_where1, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)
# where = "ID = '2'"
# gdal.Warp(fn_clip_where2, fn_in, cutlineDSName=fn_poly, cutlineLayer='polygon', cropToCutline=True, cutlineWhere=where)

In [ ]:
# https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html

# imports
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

# filepaths
fp=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip.tif'
out_tif=r'/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_merge_clip_fire.tif'

# plot raster
data=rasterio.open(fp)

In [ ]:
show((data,4),cmap='terrain')

In [ ]:
type(data)

In [ ]:
# try to insert geometry into a geodataframe

polygon='/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/perimeters_sm/santa_monica_fire_perimeters_le2018_merge.geojson'

geo=gpd.GeoDataFrame({'geometry':polygon},index=[0],crs=from_epsg(4326))

In [ ]:
# try to re-project into the same coordinate system as the raster data

geo = geo.to_crs(crs=data.crs.data)

In [ ]:
# get coordinates of the geometry in a format that rasterio wants them
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
coords = getFeatures(geo)
print(coords)

In [ ]:
# Now we are ready to clip the raster with the polygon 
# using the coords variable that we just created. 
# Clipping the raster can be done easily with the mask function 
# that we imported in the beginning from rasterio, and specifying clip=True.

out_img, out_transform = mask(raster=data, shapes=coords, crop=True)

## [Extrating Patches from Large Images and Masks for Semantic Segmentation](https://www.youtube.com/watch?v=7IL7LKSLb9I)

Following this tutorial to convert my large fire/nofire images into patches for neural network analysis. The code block below is from this video, with some alterations to adapt it to my use case.

In [41]:
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify
import tifffile as tiff

# large_image_stack_fire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif')

large_image_stack_test=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/m_3411849_se_11_060_20180722.tif')

In [40]:
# updated 
# https://stackoverflow.com/questions/68224588/problem-when-using-patchify-library-to-create-patches

import cv2

# read large_image_stack_test
img = cv2.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/m_3411849_se_11_060_20180722.tif')
# type(img)
# arr_shape=np.array(img.shape)
# print(arr_shape)
# window_shape=np.array((128,128,3))
# print(window_shape)
patches_img = patchify(img, (128,128,3), step=128)

for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_img = patches_img[i, j, 0, :, :, :]
        if not cv2.imwrite('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/fire_patches' + 'image_' + '_'+ str(i)+str(j)+'.jpg', single_patch_img):
            raise Exception("Could not write the image")

In [9]:
for img in range(large_image_stack_test.shape[0]):
    
    large_image=large_image_stack_test[img]
    
    patches_img=patchify(large_image,(128,128),step=128) # step=256 for 256 patches means no overlap
    
    for i in range(patches_img.shape[0]):
        for j in range(patches_img.shape[1]):
            
            single_patch_img=patches_img[i,j,:,:]
            tiff.imwrite('patches/images/'+'image_'+'str(img)'+'_'+str(i)+str(j)+".tif",single_patch_img)
            
            
# for msk in range(large_image_stack_test.shape[0]):
    
#     large_mask=large_mask_stack[msk]
    
#     patches_mask = patchify(large_mask,(64,64),step=64) # same guidance as above
    
#     for i in range(patches_mask.shape[0]):
#         for j in range(patches_mask.shape[1]):

#             single_patch_mask=patches_mask[i,j,:,:]
#             tiff.imwrite('patches/masks'+'mask'+str(msk)+'_'+str(i)+str(j)+".tif",single_patch_mask)

ValueError: `window_shape` is too large

In [ ]:
for img in range(large_image_stack_fire.shape[0]):
    
    large_image=large_image_stack_fire[img]
    
    patches_img=patchify(large_image,(256,256),step=256) # step=256 for 256 patches means no overlap
    
    for i in range(patches_img.shape[0]):
        for j in range(patches_img.shape[1]):
            
            single_patch_img=patches_img[i,j,:,:]
            tiff.imwrite('patches/images/'+'image_'+'str(img)'+'_'+str(i)+str(j)+".tif",single_patch_img)
            
            
for msk in range(large_image_stack_fire.shape[0]):
    
    large_mask=large_mask_stack[msk]
    
    patches_mask = patchify(large_mask,(256,256),step=256) # same guidance as above
    
    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_mask=patches_mask[i,j,:,:]
            tiff.imwrite('patches/masks'+'mask'+str(msk)+'_'+str(i)+str(j)+".tif",single_patch_mask)

In [ ]:
# import numpy as np
# from matplotlib import pyplot as plt
# from patchify import patchify
# import tifffile as tiff

# large_image_stack_nofire=tiff.imread('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_nofire.tif')



In [ ]:
# from ChatGPT
# how do you split into patches a geotiff image?

# To split a GeoTIFF image into patches, you can use a combination of GIS software and programming tools. Here's a general workflow you could follow:

# Open the GeoTIFF image in a GIS software, such as QGIS or ArcGIS.
# Determine the size of the patches you want to create. This will depend on the resolution of the image and the purpose of the patches. For example, you may want to create patches that are 256x256 pixels for use in deep learning models.
# Use the GIS software to create a grid of polygons that covers the extent of the image. The size of the polygons should match the patch size you determined in step 2.
# Export the grid of polygons as a shapefile or other vector format.
# Use a programming language, such as Python, to loop through each polygon in the shapefile and extract the corresponding patch from the GeoTIFF image using a raster processing library, such as GDAL or Rasterio.
# Save each patch as a separate image file, such as a JPEG or PNG.
# Here's some sample Python code that uses the Rasterio library to extract patches from a GeoTIFF image based on a shapefile grid:
# (code below)
# Note that this is just one possible approach, and the specific details of the workflow will depend on your particular use case and the GIS and programming tools you have available.

# this block is testing on a smaller image
# m_3411849_se_11_060_20180722.tif

# import rasterio
# import geopandas as gpd
# import os

# # Set file paths
# tif_file = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/m_3411849_se_11_060_20180722.tif"
# shp_file = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/grid/grid_spacing_100m.geojson"
# out_folder = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018_retile_fire"

# # Open the GeoTIFF image
# with rasterio.open(tif_file) as src:

#     # Open the shapefile grid
#     grid = gpd.read_file(shp_file)

#     # Loop through each polygon in the grid
#     for idx, row in grid.iterrows():

#         # Extract the polygon geometry as a GeoJSON-like object
#         geom = row.geometry.__geo_interface__

#         # Use Rasterio to read the patch as a numpy array
#         patch = src.read(window=rasterio.features.geometry_mask([geom], 
#                    out_shape=src.shape, transform=src.transform, invert=True))

#         # Set the output file path
#         out_path = os.path.join(out_folder, f"patch_{idx}.png")

#         # Save the patch as a PNG image
#         rasterio.plot.show(patch, cmap='gray', title=f"Patch {idx}")
#         plt.savefig(out_path, dpi=300)

In [ ]:
# # fullsize
# # remember to do the nofire image too

# import rasterio
# import geopandas as gpd
# import os

# # Fire
# # Set file paths
# tif_file = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018/ortho_2018_sm_fire.tif"
# shp_file = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/grid/grid_spacing_100m.geojson"
# out_folder = "/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/orthoimagery/2018_retile_fire"

# # Open the GeoTIFF image
# with rasterio.open(tif_file) as src:

#     # Open the shapefile grid
#     grid = gpd.read_file(shp_file)

#     # Loop through each polygon in the grid
#     for idx, row in grid.iterrows():

#         # Extract the polygon geometry as a GeoJSON-like object
#         geom = row.geometry.__geo_interface__

#         # Use Rasterio to read the patch as a numpy array
#         patch = src.read(window=rasterio.features.geometry_mask([geom], 
#                    out_shape=src.shape, transform=src.transform, invert=True))

#         # Set the output file path
#         out_path = os.path.join(out_folder, f"patch_{idx}.png")

#         # Save the patch as a PNG image
#         rasterio.plot.show(patch, cmap='gray', title=f"Patch {idx}")
#         plt.savefig(out_path, dpi=300)

In [ ]:
# https://gis.stackexchange.com/questions/221671/splitting-tif-image-into-several-tiles

# import os, gdal

# in_path = 'C:/Users/Marco/Desktop/'
# input_filename = 'dtm_5.tif'

# out_path = 'C:/Users/Marco/Desktop/output_folder/'
# output_filename = 'tile_'

# tile_size_x = 50
# tile_size_y = 70

# ds = gdal.Open(in_path + input_filename)
# band = ds.GetRasterBand(1)
# xsize = band.XSize
# ysize = band.YSize

# for i in range(0, xsize, tile_size_x):
#     for j in range(0, ysize, tile_size_y):
#         com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(in_path) + str(input_filename) + " " + str(out_path) + str(output_filename) + str(i) + "_" + str(j) + ".tif"
#         os.system(com_string)